# BTW 2025 Data Science Challenge

**TODO Introduction**

## Utils


In [ ]:
#from base_model import BaseModel
import torch
import numpy as np
import pandas as pd

import os

!pip install git+https://github.com/amazon-science/chronos-forecasting.git
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121

from chronos import ChronosPipeline

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-tlb5hio8
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-tlb5hio8
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit ad410c9c0ae0d499aeec9a7af09b0636844b6274
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu121


In [18]:

# TODO Load Data
# TODO Base Model Class
"""
Base model structure.
Every Forecasting model should inherit from this.

Note: override all abstract methods and keep the final methods unchanged
"""
import pandas as pd
from abc import ABC, abstractmethod
from typing import final


class BaseModel(ABC):

    def __init__(self, model_name: str, model_type: str):
        """Init and create model.

        :param model_name: Name of your model
        :param model_type: Type of your model e.g. LSTM
        """
        self.model_name = model_name
        self.model_type = model_type
        self.model = None  # this is a placeholder for your model
        self.__create_model()

    @abstractmethod
    def __create_model(self):
        """Define your own model under self.model.
        """
        ...

    @abstractmethod
    def train(self, X_train: pd.DataFrame, y_train: pd.DataFrame,
              X_val: pd.DataFrame = None, y_val: pd.DataFrame = None,
              X_test: pd.DataFrame = None, y_test: pd.DataFrame = None) -> pd.DataFrame | None:
        """train the model on the training data.
        test and validation data can be used only for evaluation (if available).

        :param X_train: training features dataset
        :param y_train: training target values
        :param X_val: validation features' dataset
        :param y_val: validation target values
        :param X_test: testing features' dataset
        :param y_test: testing target values
        :return: training history (losses while training, if available else None) [epoch | train_loss | test_loss]
        """
        # call the training loop/function of your model
        # and return a history (if available, otherwise None)
        ...

    @abstractmethod
    def __run_prediction(self, X: pd.DataFrame) -> pd.DataFrame:
        """run prediction on your defined model

        :param X: features dataset
        :return: prediction output, [timestamp | value]
        """
        ...

    @final
    def predict(self, X: pd.DataFrame, exp_dir: str = None) -> pd.DataFrame:
        """call this to run prediction

        :param X: features dataset
        :param exp_dir: dir to store prediction result
        :return: prediction output, [timestamp | value]
        """
        # run your custom prediction
        prediction_results = self.__run_prediction(X)

        # store if dir is provided
        if exp_dir is not None:
            prediction_results.to_csv(f'{exp_dir}\\{self.model_type}_{self.model_name}_prediction.csv')
        return prediction_results

    @abstractmethod
    def __custom_save(self, model: object, filename: str):
        """Use your own dataformat to save your model here

        :param filename: filename or path
        """
        ...

    @abstractmethod
    def __custom_load(self, filename: str) -> object:
        """Use your own dataformat to load your model here

        :param filename: filename or path
        :return: your loaded model
        """
        # return model
        ...

    @final
    def save(self, exp_dir: str):
        """call this to save self.model.

        :param exp_dir: dir name or path to dir
        """
        self.__custom_save(model=self.model, filename=f'{exp_dir}\\{self.model_type}_{self.model_name}')

    @final
    def load(self, exp_dir: str):
        """call this to load a retrained model

        :param exp_dir: dir name or path to dir
        """
        self.model = self.__custom_load(filename=f'{exp_dir}\\{self.model_type}_{self.model_name}')


# TODO All Models

class ChronosModel(BaseModel):
    def __init__(self, model_name: str, model_type: str):
        """Call the BaseModel constructor with the required arguments."""
        super().__init__(model_name, model_type)

    def _BaseModel__create_model(self):
        self.model = None

    
    def _BaseModel__run_prediction(self, X):

        #define which column to be forecasted and forecast legth
        target_column = "day_ahead_prices_EURO"
        prediction_length = 24

        
        context = torch.tensor(X[target_column].values)[-512:]  # Limit context to last 512 samples
        forecast = self.model.predict(context, prediction_length)
        low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)

        context_dates = X.index[-512:]
        last_date = context_dates[-1]
        forecast_index = pd.date_range(last_date + pd.Timedelta(hours=1), periods=prediction_length, freq="H")

        prediction_results = pd.DataFrame({
            "timestamp": forecast_index,
            "forecasted_values": median
        })
        
        return prediction_results

    
    def _BaseModel__custom_load(self, filename):
        # Directory to clone
        github_repo_url = filename
        local_dir = "./final-submission"

        # Clone the repository if it doesn't already exist
        if not os.path.exists(local_dir):
            os.system(f"git clone {github_repo_url} {local_dir}")

        # Path to the specific directory containing the checkpoint
        checkpoint_dir = os.path.join(local_dir, "chronos/models/models/Chronos-Tiny-2015-1000/checkpoint-final")
        
        # Load the model pipeline
        pipeline = ChronosPipeline.from_pretrained(
            checkpoint_dir,
            device_map=("cuda" if torch.cuda.is_available() else "cpu"),
            torch_dtype=torch.bfloat16,
        )

        return pipeline

    def _BaseModel__custom_save(self, model = None, filename = None):
        return

    def train(self, X_train = None, y_train = None, X_val = None, y_val = None, X_test = None, y_test = None):
        return None


In [ ]:
model_name = "Chronos-Tiny"
model_type = "LSTM"  # Or any model type you prefer
model = ChronosModel(model_name=model_name, model_type=model_type)
repo_url = "git@github.com:BTW25-Data-Science-Challenge/final-submission.git"
model.model = model._BaseModel__custom_load(repo_url)

# Simulate input data (example DataFrame)
data = pd.read_csv("/home/julius/final-submission/data/day_ahead_prices.csv", parse_dates=["timestamp"]).set_index("timestamp")

# Perform a forecast
forecast_results = model._BaseModel__run_prediction(data)

# Display the forecast
print(forecast_results)

Cloning into './final-submission'...


## Gathering Domain Knowledge 

**TODO What does the reader have to know about the energy market**

## Data Sources

## Data Cleaning


## Data Analysis

### Comparing Feature Importances

#### AutoGluon

#### Temporal-Fusion-Transformer

## Visualization & Story Telling

### Baseline Models Benchmark

### LSTMs

### Chronos

Chronos is a framework for pre-trained probabilistic time series models introduced by TODO Chronos in March 2024. It tokenizes time series values into a fixed vocabulary through scaling and quantization and trains transformer-based language models on these tokens using the cross-entropy loss function. This means that from a time series after a mean scaling values are taken from defined points of a time series. These context tokens are then used for the (pre)training. Chronos is designed without time-series-specific architecture, resulting in a minimalistic yet effective approach. The framework achieved remarkable results in in-domain experiments and demonstrated competitive zero-shot performance, comparable to models specifically trained on similar tasks.

The developer of Chronos provided a GitHub Repository (https://github.com/amazon-science/chronos-forecasting/tree/main) which enables the user to either use the pretrained models for forecasting fine tune models on their own Data. Compared to the T5 architecture Chronos reduces the vocabulary size resulting in five different models ranging from eight million to 710 million parameters.

In this work, the pretrained Chronos-T5 (Tiny) model was utilized as a benchmark to assess and improve its performance by fine-tuning it on our own data. Specifically, we focused on fine tune the model with the day-ahead electricity prices from ENTSO-E as a domain-specific dataset. The tiny model was chosen for its practicality, as it can be fine-tuned and utilized for forecasting tasks even on a standard laptop. To gain deeper insights into the impact of the dataset size and training steps on model performance, we conducted fine-tuning experiments in four distinct ways.

The energy market has experienced heightened volatility in recent years, driven by geopolitical and economic disruptions such as the Ukraine war. To evaluate the impact of dataset characteristics on model performance, we divided the data into two subsets. The first dataset contains day-ahead prices from January 2022 to December 2023, and the second one data spanning from January 2015 to December 2023. The smaller dataset focuses primarily on recent, highly volatile market conditions, reflecting current dynamics. In contrast, the larger dataset spans a longer historical period, capturing a broader range of market scenarios. This approach enables a direct comparison to determine whether the smaller, more focused dataset enhances adaptability to recent volatility or if the larger dataset provides a more comprehensive foundation due to its diversity.

The developers of Chronos fine-tuned their Chronos-T5 (Small) model with 1000 training steps and achieved remarkable results. We were interested if even more fine-tuning steps could increase the models performance. Therefore, for both datasets, the fine-tuning was executed with two configurations of training steps—1,000 and 10,000 steps. This comprehensive setup aimed to explore the potential of domain-specific fine-tuning in enhancing the model's capabilities.

Chronos models are probabilistic models. Therefore an evaluation which of the fine tuned models performs the best should be done on a large test-set. To determine which configuration performs best for the challenge of predicting 24-hour day-ahead energy prices, we conducted forecasts for a full year, from the beginning of December 2023 to the end of November 2024. For each forecasted day, the context data consisted of the most recent 512 hourly day-ahead prices.

The results are computes as following. For each 24 values of one day we calculated the root mean squared error and the absolute error. For each day the mean of these values are calculated and over the whole year the mean is created again. The yearly Results of the Chronos-T5 (Tiny) model and its fine-tuned versions are presented in the table below.

|Chronos-T5 (Tiny)    |1. Zero-Shot  |2. Fine-Tuned Data: 2015 Steps: 1000|3. Fine-Tuned Data: 2015 Steps: 1000|4. Fine-Tuned Data: 2015 Steps: 1000|5. Fine-Tuned Data: 2015 Steps: 1000|
|------------|------------|------------|------------|------------|------------|
| RMSE              | 25.817  | 22.739    | 23.501 | 24.552 | 26.413 |
| RMSE in percent   | 117.48% | 120.92%   | 146.55% | 128.25% | 137.28% |
| MAE               | 20.127  | 17.254    | 17.969 | 18.59 | 20.263 |
| MAE in percent    | 2637.80% | 2484.01% | 3311.03% | 3168.22% | 4038.85% |

These results reveal several key insights. First, not all fine-tuned models outperform the benchmark. Additionally, models fine-tuned on the larger dataset, which includes data from 2015, tend to perform better than those fine-tuned on the smaller, more recent dataset. This may be due to the fact that, although the energy market is currently highly volatile, less volatile days dominate the market, making the larger dataset, with its broader range of scenarios, more beneficial. Furthermore, the models trained with 1,000 steps, rather than 10,000, tend to perform better. This could indicate that overfitting occurred with the longer training duration.

It is clear that the model which was fine-tuned on the large dataset with 1000 training steps Performs the best. This model has the lowest RMSE and MAE. On average the difference between the predicted hourly day-ahead price and its actual value is 17.254 Euro.

Interesting is that model three and four have still a better performance in the MAE and RMSE, but its corresponding percentage value are outperformed by the benchmark. This shows that with the pretraining the overall error is reduced, but the models are getting worse in predicting extreme scenarios like energy prices above 500 Euros and prices below zero euros.

Model five is outperformed by the benchmark in all metrics.

As already mentioned there are several sizes of Chronos models available. Additional to the Chronos-T5 (Tiny) model we investigated the performance of the Chronos-T5 (Large) model. For this model we performed the same fine-tuning steps as we did for the hronos-T5 (Tiny) model.

The yearly Results of the Chronos-T5 (Large) model and its fine-tuned versions are presented in the table below.

|Chronos-T5 (Large)    |Zero-Shot  |Fine-Tuned Data: 2015 Steps: 1000|Fine-Tuned Data: 2015 Steps: 1000|Fine-Tuned Data: 2015 Steps: 1000|Fine-Tuned Data: 2015 Steps: 1000|
|------------|------------|------------|------------|------------|------------|
| RMSE | 25.803 | 22.818 | Row 1 Col4 | Row 1 Col5 | Row 1 Col6 |
| MAE | 2575.93% | 2604.36% | Row 2 Col4 | Row 2 Col5 | Row 2 Col6 |
| MAE in percent | 2575.93% | 2604.36% | Row 2 Col4 | Row 2 Col5 | Row 2 Col6 |

In [6]:
!pip install git+https://github.com/amazon-science/chronos-forecasting.git
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-776qxked
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-776qxked
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit ad410c9c0ae0d499aeec9a7af09b0636844b6274
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 KB 1.4 MB/s eta 0:00:0000:0100:01
  Created wheel for chronos-forecasting: filename=chronos_forecasting-1.4.1-py3-none-any.whl size=29154 sha256=0ddca36d3ab7c326aa669577503c52cb129ae029cab17162867aeeda02cdaa08
  Stored in directory: /tmp/pip-ephem-wheel-cache-b97xhgwz/wheels/bf/c1/65/08857e57345ef1b34ba0edb4791f1b2594943e82f34e93a4ab
Successfully built chronos-forecas

In [10]:
!wget https://github.com/BTW25-Data-Science-Challenge/final-submission/tree/chronos/models/models/Chronos-Tiny-2015-1000/checkpoint-final

--2025-01-16 15:15:35--  https://github.com/BTW25-Data-Science-Challenge/final-submission/tree/chronos/models/models/Chronos-Tiny-2015-1000/checkpoint-final
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 

404 Not Found
2025-01-16 15:15:35 ERROR 404: Not Found.



### Temporal Fusion Transformer

### AutoGluon

### Hacking AutoGluon

## Predictive Modeling

### Final Benchmark

### Final Forecast

In [3]:
# TODO Download Up to date Data
# TODO Load best model from disk
# TODO Compute Forecast
# TODO Write to File

## Summary

## Future Work

To further improve Chronos one option would be to do a hyperparameter optimization. This would help to find the best hyperparameter configuration for a specific model. 

## Conclusion 